# 📌 Intro.

- 구글이미지에 각 POI의 장소명을 입력해 이미지 검색후 크롤링
- AI hub에서 제공하는 데이터셋의 이미지 품질이 저조하여 신규 데이터 수집을 수행

## 필요 라이브러리 선언

In [ ]:
# 웹 드라이버 및 자동 설치 관련
import chromedriver_autoinstaller
import edgedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 웹 요청 및 이미지 처리 관련
import requests
from PIL import Image
from io import BytesIO

# 사용자 에이전트 생성 관련
from fake_useragent import UserAgent

# 기타 유틸리티
import os
import gc
import time
import ssl
import re
import urllib.request
from tqdm.notebook import tqdm
import pandas as pd

## 크롤링 함수 작성

In [ ]:
# 크롬 버전 확인 및 드라이버 설정
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]

# SSL 인증을 위한 설정
ssl._create_default_https_context = ssl._create_default_https_context

def create_driver():
    """
    Chrome 드라이버를 생성하는 함수
    - fake_useragent를 사용하여 랜덤한 User-Agent 설정
    - 창을 화면 밖으로 이동시키는 옵션 설정
    """
    
    # fake_useragent를 이용하여 랜덤한 UserAgent를 생성
    ua = UserAgent()
    user_agent = ua.random

    # Chrome 드라이버의 옵션을 설정
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    #options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # 창을 화면 밖으로 이동시키는 옵션
    options.add_argument('--window-position=10000,0')

    # 설정한 옵션으로 Chrome 드라이버를 생성
    driver = webdriver.Chrome(options=options)
    return driver

def scroll_down(driver):
    """
    웹 페이지를 아래로 스크롤하는 함수
    - 스크롤을 내리면서 새로운 이미지를 로드
    - 더 이상 로드할 이미지가 없거나 스크롤이 더 이상 내려가지 않을 때까지 반복
    """
    
    SCROLL_PAUSE_TIME = 1  # 페이지 스크롤 대기시간 
    last_height = driver.execute_script("return document.body.scrollHeight")
    new_height = 0
    
    while True:
        # 페이지 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 스크롤 후 로딩 대기
        time.sleep(SCROLL_PAUSE_TIME)
        
        # 스크롤 후 페이지 높이 다시 저장
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 새로운 높이와 이전 높이가 같으면 스크롤 종료
        if new_height == last_height:
            try:
                # 더보기 버튼 클릭
                driver.find_element(By.CSS_SELECTOR, ".mye4qd").click()
                time.sleep(SCROLL_PAUSE_TIME)
                new_height = driver.execute_script("return document.body.scrollHeight")
                
                # 더보기 버튼 클릭 후에도 높이 변화가 없으면 스크롤 종료
                if new_height == last_height:
                    break
                else:
                    last_height = new_height
            except:
                # 더보기 버튼이 없거나 스크롤이 더 이상 내려가지 않으면 스크롤 종료
                break
        else:
            last_height = new_height

def download_images(driver, search_key, folder_name, start_count):
    ua = UserAgent()
    images = driver.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
    count = start_count

    for image in images:
        if count > 35:
            break

        try:
            image.click()
            time.sleep(0.5)

            imgUrl = driver.find_element(
                By.XPATH,
                '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'
            ).get_attribute("src")

            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', ua.random)]
            urllib.request.install_opener(opener)
            
            # SSL 인증서 검증 우회
            ssl._create_default_https_context = ssl._create_unverified_context
            
            urllib.request.urlretrieve(imgUrl, os.path.join(folder_name, f'{search_key}_{str(count)}.jpg'))
            count += 1
        except:
            print(f"Failed to download image. Skipping...")
            raise
    
    return count

def google_image_crawler(search_key, region_name):
    region_dict = {
        'jeju': '제주',
        'gyeonggi': '경기',
        'gangwon': '강원',
        'chungbuk': '충북',
        'chungnam': '충남',
        'gyeongbuk': '경북',
        'gyeongnam': '경남',
        'jeonbuk': '전북',
        'jeonnam': '전남',
        'seoul': '서울',
        'busan': '부산',
        'incheon': '인천',
        'gwangju': '광주',
        'daejeon': '대전',
        'ulsan': '울산',
        'sejong': '세종'
    }

    """
    Google 이미지를 크롤링하는 메인 함수
    - 입력 키워드 전처리
    - 저장 경로 설정
    - 기존 작업 유무 체크
    - 검색 및 이미지 다운로드 수행
    """
    
    """
    입력 키워드 전처리
    """
    # 특수 문자가 포함된 경우에만 대체하고, 특수 문자가 없는 경우는 입력값 유지
    if re.search(r'[\\/:*?"<>|]', search_key):
        search_key = re.sub(r'[\\/:*?"<>|]', '_', search_key)
    
    """
    저장 경로 설정
    """
    folder_name = f'./data/travel_imgs/{region_name}/{search_key}'
    start_count = 1  # 다운로드 시작번호 설정

    # 상위 디렉토리 생성
    os.makedirs(f'./data/travel_imgs/{region_name}', exist_ok=True)
    
    """
    기존 작업 유무 체크
    """
    # 파일 수 확인 및 조기 반환 로직
    if os.path.exists(folder_name):  # 키워드명으로된 폴더 있는 경우
        file_list = os.listdir(folder_name)
        file_count = len(file_list)
        if file_count >= 35:
            print(f"{search_key} 폴더에 이미 35개의 이미지가 있습니다. 다음 키워드로 넘어갑니다.")
            return  # 함수종료 (크롤링 하지 않음)
        
        # 폴더는 있지만 이미지가 35개 이하인 경우 : 크롤링 필요
        else:
            start_count = file_count + 1  # 기존 파일개수를 기준으로 신규 파일명의 번호 기준점 set
    # 키워드 명으로된 폴더가 없는 경우 생성
    else:
        os.makedirs(folder_name)
    
    """
    검색 시작
    """
    # region_dict에서 region_name에 해당하는 한글 값 가져오기
    region_korean = region_dict.get(region_name)
    
    if region_korean:
        search_query = region_korean + ' ' + search_key + ' 여행'
    else:
        print(f"잘못된 region_name입니다: {region_name}")
        return
    
    # 파일 수가 35개 미만인 경우에만 웹 드라이버 생성 및 검색 수행
    while True:
        driver = create_driver()
        driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi&ogbl")
        elem = driver.find_element("name", "q")
        elem.send_keys(search_query)
        elem.send_keys(Keys.RETURN)

        try:
            while True:
                scroll_down(driver)
                
                # 30초 동안 이미지 수집이 되지 않는 경우 재시도
                timeout = time.time() + 30
                current_count = start_count
                while current_count == start_count:
                    current_count = download_images(driver, search_key, folder_name, start_count)
                    if time.time() > timeout:
                        raise Exception("Timeout: Image download not progressing")
                
                if current_count > 35:
                    break
                start_count = current_count
        except Exception as e:
            print(f"Error occurred: {e}. Restarting the browser and retrying.")
            driver.quit()
            
            # 에러 발생 시 폴더 존재 여부와 파일 개수 확인
            if os.path.exists(folder_name):
                file_list = os.listdir(folder_name)
                file_count = len(file_list)
                if file_count >= 35:
                    print(f"{search_key} 폴더에 이미 35개의 이미지가 있습니다. 다음 키워드로 넘어갑니다.")
                    break
                else:
                    start_count = file_count + 1
            else:
                os.makedirs(folder_name)
                start_count = 1
            
            continue
        
        driver.quit()
        break
    
    # 크롤링 결과 출력
    file_list = os.listdir(folder_name)
    file_count = len(file_list)
    print(f"{search_key} 키워드로 크롤링 완료. 총 {file_count}개의 이미지를 수집하였습니다.")

## 크롤링 목록 추출 리스트 생성

In [ ]:
filtered_df = pd.read_csv('../data/tour_api/filtered_tour_meta.csv',encoding = 'utf-8-sig')

In [ ]:
filtered_df.head()

In [ ]:
filtered_df.info()

## 크롤링 수행

- 하나의 keyword 당 30장의 이미지를 성공적으로 수집했다면 다음 keyword로 넘어가도록 합니다.

## 제주지역

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
jeju_df = filtered_df[filtered_df['addr1'].str.contains("제주", na=False)]
jeju_df.info()

In [ ]:
jeju_df.to_csv('jeju_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 제주 지역에 대한 이미지만 수집
search_keywords = jeju_df['title'][:-28] # 제주지역 마지막 27개는 올레길이라 이미지수집이 무의미 (코스이미지가 대부분)


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'jeju')

## 경기도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
gyeonggi_df = filtered_df[filtered_df['addr1'].str.contains("경기", na=False)]
gyeonggi_df.info()

In [ ]:
gyeonggi_df.to_csv('gyeonggi_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경기 지역에 대한 이미지만 수집
search_keywords = gyeonggi_df['title'][:-95] # 여행길 혹은 코스로된 부분 삭제


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'gyeonggi')

## 강원도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
gangwon_df = filtered_df[filtered_df['addr1'].str.contains("강원", na=False)]
gangwon_df.info()

In [ ]:
gangwon_df.to_csv('gangwon_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 강원 지역에 대한 이미지만 수집
search_keywords = gangwon_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'gangwon')

## 충청북도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
chungbuk_df = filtered_df[filtered_df['addr1'].str.contains("충청북도", na=False)]
chungbuk_df.info()

In [ ]:
chungbuk_df.to_csv('chungbuk_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 충청북도 지역에 대한 이미지만 수집
search_keywords = chungbuk_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'chungbuk')

## 충청남도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
chungnam_df = filtered_df[filtered_df['addr1'].str.contains("충청남도", na=False)]
chungnam_df.info()

In [ ]:
chungnam_df.to_csv('chungnam_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 충청남도 지역에 대한 이미지만 수집
search_keywords = chungnam_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'chungnam')

## 경상북도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
gyeongbuk_df = filtered_df[filtered_df['addr1'].str.contains("경상북도", na=False)]
gyeongbuk_df.info()

In [ ]:
gyeongbuk_df.to_csv('gyeongbuk_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경상북도 지역에 대한 이미지만 수집
search_keywords = gyeongbuk_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'gyeongbuk')

## 경상남도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
gyeongnam_df = filtered_df[filtered_df['addr1'].str.contains("경상남도", na=False)]
gyeongnam_df.info()

In [ ]:
gyeongnam_df.to_csv('gyeongnam_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경상남도 지역에 대한 이미지만 수집
search_keywords = gyeongnam_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'gyeongnam')

## 전라북도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
jeonbuk_df = filtered_df[filtered_df['addr1'].str.contains("전북|전라북", na=False)]
jeonbuk_df.info()

In [ ]:
jeonbuk_df.to_csv('jeonbuk_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경기 지역에 대한 이미지만 수집
search_keywords = jeonbuk_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'jeonbuk')

## 전라남도

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
jeonnam_df = filtered_df[filtered_df['addr1'].str.contains("전남|전라남", na=False)]
jeonnam_df.info()

In [ ]:
jeonnam_df.to_csv('jeonnam_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경기 지역에 대한 이미지만 수집
search_keywords = jeonnam_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'jeonnam')

## 서울특별시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
seoul_df = filtered_df[filtered_df['addr1'].str.contains("서울", na=False)]
seoul_df.info()

In [ ]:
seoul_df.to_csv('seoul_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 서울 지역에 대한 이미지만 수집
search_keywords = seoul_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'seoul')

## 부산광역시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
busan_df = filtered_df[filtered_df['addr1'].str.contains("부산", na=False)]
busan_df.info()

In [ ]:
busan_df.to_csv('busan_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 부산 지역에 대한 이미지만 수집
search_keywords = busan_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'busan')

## 인천광역시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
incheon_df = filtered_df[filtered_df['addr1'].str.contains("인천", na=False)]
incheon_df.info()

In [ ]:
incheon_df.to_csv('incheon_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 인천 지역에 대한 이미지만 수집
search_keywords = incheon_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'incheon')

## 광주광역시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
gwangju_df = filtered_df[filtered_df['addr1'].str.contains("광주광", na=False)]
gwangju_df.info()

In [ ]:
gwangju_df.to_csv('gwangju_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 광주광 지역에 대한 이미지만 수집
search_keywords = gwangju_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'gwangju')

## 대전광역시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
daejeon_df = filtered_df[filtered_df['addr1'].str.contains("대전", na=False)]
daejeon_df.info()

In [ ]:
daejeon_df.to_csv('daejeon_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 대전 지역에 대한 이미지만 수집
search_keywords = daejeon_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'daejeon')

## 울산광역시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
ulsan_df = filtered_df[filtered_df['addr1'].str.contains("울산", na=False)]
ulsan_df.info()

In [ ]:
ulsan_df.to_csv('ulsan_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경기 지역에 대한 이미지만 수집
search_keywords = ulsan_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'ulsan')

## 세종특별자치시

In [ ]:
# null 값을 처리하여 'contains' 메서드 사용
sejong_df = filtered_df[filtered_df['addr1'].str.contains("세종", na=False)]
sejong_df.info()

In [ ]:
sejong_df.to_csv('sejong_db.csv',encoding='utf-8-sig', index=False)

In [ ]:
# 검색어 리스트
# 샘플로 경기 지역에 대한 이미지만 수집
search_keywords = sejong_df['title']


print(len(search_keywords),"\n")

search_keywords

In [ ]:
# 각 검색어에 대해 크롤링 수행
for keyword in tqdm(search_keywords, desc='이미지 크롤링 진행 중'):
    print('-'*100)
    print(f"{keyword} 수집시도")
    google_image_crawler(keyword, 'sejong')